In [ ]:
import astropy
from astropy.table import Table, join, vstack
from astropy.time import Time
from astropy.io import fits
from astropy import units as u

from astroquery.jplhorizons import Horizons

import numpy as np
import matplotlib.pyplot as plt
from io import BytesIO, StringIO
import time
import requests
import sys
import os
import getpass

saveplots = False # set to save some of the plots to png files
import pandas as pd
from astropy.time import Time
from astropy.coordinates import SkyCoord
from urllib.parse import quote

# Functions

In [ ]:
def cadc_ssos_query(object_name, search="bynameall", 
                    # epoch1=50000, epoch2=57079,
                    xyres="no", 
                    # telinst="Pan-STARRS1", 
                    lang="en", format="tsv",
                    url="https://www.cadc-ccda.hia-iha.nrc-cnrc.gc.ca/cadcbin/ssos/ssosclf.pl"):
    """Use CADC moving object query to find PS1 observations by target name
    
    The epoch parameters give the range in MJD for the PS1 observations.
    The only parameter that might be usefully modified is the search.  
    E.g., use search="bynameHorizons" to search using the JPL Horizons emphemeris rather
    than the ephemeris cached at CADC.  That might be useful if the CADC cache has become
    out-of-date.
    
    This return an astropy table with the observations.
    """
    t0 = Time('1990-01-01 00:00:00', format='iso', scale='utc')
    t1 = Time.now()

    print(f"Querying CADC SSOS for object '{object_name}' between {t0.iso} and {t1.iso}")
    params = dict(
        lang=lang,
        object=object_name,
        search=search,
        epoch1=t0.mjd,
        epoch2=t1.mjd,
        format=format
    )

    try:
        r = requests.get(url, params=params, timeout=60)
    except requests.exceptions.RequestException as e:
        print(f"HTTP request failed: {e}")
        # Return an empty table with expected columns so downstream code doesn't crash
        cols = ['Image', 'MJD', 'Filter', 'Exptime', 'Object_RA', 'Object_Dec',
                'Image_target', 'Telescope/Instrument', 'MetaData', 'Datalink']
        return Table.from_pandas(pd.DataFrame(columns=cols))

    print(r.url)

    text = r.text
    first_line = text.strip().split('\n', 1)[0]

    # Detect common error responses from CADC / Horizons and HTML error pages
    if (first_line.lower().startswith("there was an error") or
        "error" in first_line.lower() or
        first_line.strip().startswith("<!DOCTYPE") or
        first_line.strip().startswith("<html")):
        # Clean up message for printing
        msg = first_line.replace('<br/>', '').strip()
        print(f"CADC/SSOS returned an error: {msg}")
        # Try a simple fallback: switch to CADC cached ephemerides if not already using it
        if search != "bynameCADC":
            print("Retrying with search='bynameCADC'...")
            return cadc_ssos_query(object_name, search="bynameCADC", xyres=xyres, lang=lang, format=format, url=url)
        # Otherwise return an empty table with expected columns
        cols = ['Image', 'MJD', 'Filter', 'Exptime', 'Object_RA', 'Object_Dec',
                'Image_target', 'Telescope/Instrument', 'MetaData', 'Datalink']
        return Table.from_pandas(pd.DataFrame(columns=cols))

    # Parse TSV using pandas to avoid astropy fast C-reader converter limitations
    try:
        df = pd.read_csv(StringIO(text), sep='\t', engine='python', on_bad_lines='skip')
    except Exception as e:
        print(f"Failed to parse CADC TSV: {e}")
        # Return empty table with expected columns to keep pipeline running
        cols = ['Image', 'MJD', 'Filter', 'Exptime', 'Object_RA', 'Object_Dec',
                'Image_target', 'Telescope/Instrument', 'MetaData', 'Datalink']
        return Table.from_pandas(pd.DataFrame(columns=cols))

    # Ensure expected columns exist even if CADC response changes
    expected_cols = ['Image', 'MJD', 'Filter', 'Exptime', 'Object_RA', 'Object_Dec',
                     'Image_target', 'Telescope/Instrument', 'MetaData', 'Datalink']
    for col in expected_cols:
        if col not in df.columns:
            df[col] = pd.NA

    # Ensure string type for Image_target if present
    if 'Image_target' in df.columns:
        df['Image_target'] = df['Image_target'].astype(str)

    return Table.from_pandas(df)



In [ ]:
# Define a simple ephemerides query
def ephemcc(ident, ep, nbd=None, step=None, observer='645', rplane='1', tcoor=5):
    '''Gets asteroid ephemerides from IMCCE Miriade for a suite of JD for a single SSO
    Original function by M. Mahlke

    :ident: int, float, str - asteroid identifier
    :ep: float, str, list - Epoch of computation
    :observer: str - IAU Obs code - default to geocenter: https://minorplanetcenter.net//iau/lists/ObsCodesF.html
    :returns: pd.DataFrame - Input dataframe with ephemerides columns appended
              False - If query failed somehow

    '''
    
    # ------
    # Miriade URL 
    url = 'https://ssp.imcce.fr/webservices/miriade/api/ephemcc.php'
    
    #if rplane=='2':
    #    tcoor='1'
        
    # Query parameters
    params = {
        '-name': f'c:{ident}',
        '-mime': 'json',
        '-rplane': rplane,
        '-tcoor': tcoor,
        '-output': '--jd',
        '-observer': observer, 
        '-tscale': 'UTC',
        '-type': 'Asteroid'
    }
    

    # Single epoch of computation
    if type(ep)!=list:
        # Set parameters
        params['-ep'] = ep
        if nbd!=None: 
            params['-nbd'] = nbd
        if step!=None: 
            params['-step'] = step

        # Execute query
        try:
            r = requests.post(url, params=params, timeout=80)
        except requests.exceptions.ReadTimeout:
            return False


    # Multiple epochs of computation
    else:
        # Epochs of computation
        files = {'epochs': ('epochs', '\n'.join(['%.6f' % epoch
                                                 for epoch in ep]))}

        # Execute query
        try:
            r = requests.post(url, params=params, files=files, timeout=120)
            print(r.url)
        except requests.exceptions.ReadTimeout:
            return False

    j = r.json()

    # Read JSON response
    try:
        ephem = pd.DataFrame.from_dict(j['data'])
    except KeyError:
        return False

    return ephem


In [ ]:
def decam_cleaner(df: pd.DataFrame)-> pd.DataFrame:
    """Clean up DECam observation table from CADC SSOS query

    This function removes duplicate entries and filters out non-DECam observations.
    """

    def select_decam_images(group):
        # Select only calibrated images containing '_ooi'
        cond = group['Image'].str.contains('_ooi') | group['Image'].str.contains('_opi')
        group_filtered = group[cond]
        # If both _opi and _ooi exist, prefer _opi
        if group_filtered['Image'].str.contains('_opi').any():
            group_filtered = group_filtered[group_filtered['Image'].str.contains('_opi')]
        if len(group_filtered) > 1:
            if group_filtered['Image'].str.contains('_v4').any():
                return group_filtered[group_filtered['Image'].str.contains('_v4')]
            elif group_filtered['Image'].str.contains('_v3').any():
                return group_filtered[group_filtered['Image'].str.contains('_v3')]
            elif group_filtered['Image'].str.contains('_v2').any():
                return group_filtered[group_filtered['Image'].str.contains('_v2')]
            elif group_filtered['Image'].str.contains('_v1').any():
                return group_filtered[group_filtered['Image'].str.contains('_v1')]
            else:
                return group_filtered.iloc[[0]]
        else:
            return group_filtered

    # Filter to only include DECam observations (make a copy to avoid SettingWithCopyWarning)
    df_decam = df[df['Telescope/Instrument'].str.contains('DECam', na=False)].copy()

    if df_decam.empty:
        return df

    # Remove decam images from the original dataframe
    df = df.drop(df_decam.index)

    # Further filter to only images starting with 'c4d_'
    cond = df_decam['Image'].str.startswith('c4d_')
    df_decam = df_decam[cond]

    # Recompute grouping key (assign via .loc to avoid SettingWithCopyWarning)
    df_decam.loc[:, 'date'] = df_decam['Image'].apply(lambda x: "_".join(x.split('_')[:3]))
    gr = df_decam.groupby('date')

    if gr.ngroups == 0:
        return df_decam

    # Check pandas version to use correct apply syntax
    if hasattr(pd, '__version__') and int(pd.__version__.split('.')[0]) >= 2:
        df_selected = gr.apply(select_decam_images, include_groups=False) # type: ignore
    else:
        df_selected = gr.apply(select_decam_images)

    # display(df_decam)
    # insert back decam images into the original dataframe
    df = pd.concat([df, df_selected], ignore_index=True)
    
    del df_selected, df_decam

    return df

# Telescopes visibility from Telescope limits

- This should be read for both 1. and 2. 

In [ ]:
df_limits = pd.read_csv("data/Survey-Filter-LimitingMagnitude-SaturationMagnitude-TypicalExposureTime.csv")
df_limits

# 1. List of the comets

## Read data

In [ ]:
df_init = pd.read_csv('data/List of comets.csv')
df_init

## Read tables from CADC

In [ ]:
%%time 
for i, row in df_init.iterrows():
    print(f"{i}: {row['Comet name']}")
    source = df_init.loc[i, 'Comet name']
    name = " ".join(source.split())

    cadc_tab = cadc_ssos_query(name)
    print(f"Returned table has {len(cadc_tab)} rows")
    df_comet = cadc_tab.to_pandas()
    df_comet.to_csv(f"data/cadc/{name.replace('/', '_')}_cadc.csv", index=False)
    # break



## Check Miriade

In [ ]:
rows_limit = 500
for i, row in df_init[1:].iterrows():
    print(f"{i}: {row['Comet name']}")
    source = df_init.loc[i, 'Comet name']
    name = " ".join(source.split())
    filename = f"data/cadc/{name.replace('/', '_')}_cadc.csv"
    df_obs = pd.read_csv(filename)
    
    chunks = len(df_obs) // rows_limit + 1
    print(f"  Read {len(df_obs)} observations from {filename} in {chunks} chunks")

    df_comet = None
    for chunk in range(chunks):
        start = chunk * rows_limit
        end = min((chunk + 1) * rows_limit, len(df_obs))
        jds = (df_obs['MJD'].values[start:end] + 2400000.5).tolist()
        
        ephem = ephemcc(name, jds, observer='500', rplane='1', tcoor=5)
        if ephem is False:
            print(f"  Miriade query failed for {name}")
            continue
        
        if df_comet is None:
            df_comet = ephem
        else:
            df_comet = pd.concat([df_comet, ephem], ignore_index=True)

    if df_comet is not None:
        print(f"Got {len(df_comet)} ephemeris rows from Miriade")
        df_comet.to_csv(f"data/miriade/{name.replace('/', '_')}_miriade.csv", index=False)

    

## Analysis of the results

In [ ]:
counts = 0
for i, row in df_init[0:].iterrows():
    name = " ".join(row['Comet name'].split())
    cadc_filename = f"data/cadc/{name.replace('/', '_')}_cadc.csv"
    miriade_filename = f"data/miriade/{name.replace('/', '_')}_miriade.csv"
    try:
        df_cadc = pd.read_csv(cadc_filename)
    except FileNotFoundError:
        print(f"{name}: CADC file not found")
        continue
    
    try:
        df_miriade = pd.read_csv(miriade_filename)
    except FileNotFoundError:
        print(f"{name}: Miriade file not found")
        continue
    
    df = pd.merge(df_cadc, df_miriade, left_index=True, right_index=True, suffixes=('_cadc', '_miriade'))
    print(f"{name}: merged {len(df)} rows")
    
    cond = (df['VMag'] < 18) & (df["Telescope/Instrument"] == "NEOSSat")
    df_visible = df[cond]
    counts += len(df_visible)
    # break
print(f"Total visible observations: {counts}")

In [ ]:
df_visible.value_counts('Telescope/Instrument')

In [ ]:
# for i, row in df_init[5:].iterrows():
#     name = " ".join(row['Comet name'].split())
#     cadc_filename = f"data/cadc/{name.replace('/', '_')}_cadc.csv"
#     miriade_filename = f"data/miriade/{name.replace('/', '_')}_miriade.csv"

# 2. New list of the comets

## Read data

In [ ]:
l2 = "C/2004 D1, C/2005 R4, C/2005 T3, C/2006 L1, C/2006 Q1, C/2007 N3, C/2007 R1, C/2008 FK75, C/2009 F2, C/2009 S3, C/2009 U5, C/2010 G2, C/2012 A1, C/2016 R2, C/2023 H5"
df_init = pd.DataFrame({"Comet": l2.split(", ")})
df_init.to_csv("data/comets_list2.csv", index=False)
df_init

## Get possible dates of observation from CADC

In [ ]:
%%time 
for i, row in df_init.iterrows():
    print(f"{i}: {row['Comet']}")
    source = df_init.loc[i, 'Comet']
    name = " ".join(source.split())
    cadc_filename = f"data/cadc2/{name.replace('/', '_')}_cadc.csv"

    if os.path.exists(cadc_filename):
        print(f"  File {cadc_filename} exists, skipping")
        continue

    cadc_tab = cadc_ssos_query(name)
    
    print(f"Returned table has {len(cadc_tab)} rows")
    df_comet = cadc_tab.to_pandas()
    # display(df_comet)
    
    # clean DECam entries
    # df_comet_cleaned = decam_cleaner(df_comet)
    # print(f"After DECam cleaning, {len(df_comet_cleaned)} rows remain")
    df_comet.to_csv(cadc_filename, index=False)
    # if i == 3:
    #     break


## Get ephemerides from Miriade for the selected dates

In [ ]:
rows_limit = 2000
for i, row in df_init[1:].iterrows():
    print(f"{i}: {row['Comet']}")
    source = df_init.loc[i, 'Comet']
    name = " ".join(source.split())
    cadc_filename = f"data/cadc2/{name.replace('/', '_')}_cadc.csv"
    miriade_filename = f"data/miriade2/{name.replace('/', '_')}_miriade.csv"
    if os.path.exists(miriade_filename):
        print(f"  File {miriade_filename} exists, skipping")
        continue

    df_obs = pd.read_csv(cadc_filename)
    chunks = len(df_obs) // rows_limit + 1
    print(f"  Read {len(df_obs)} observations from {cadc_filename} in {chunks} chunks")

    df_comet = None
    for chunk in range(chunks):
        start = chunk * rows_limit
        end = min((chunk + 1) * rows_limit, len(df_obs))
        jds = (df_obs['MJD'].values[start:end] + 2400000.5).tolist()
        
        ephem = ephemcc(name, jds, observer='500', rplane='1', tcoor=5)
        if ephem is False:
            print(f"Miriade query failed for {name}")
            time.sleep(1)
            continue
        
        if df_comet is None:
            df_comet = ephem
        else:
            df_comet = pd.concat([df_comet, ephem], ignore_index=True)

    if df_comet is not None:
        print(f"Got {len(df_comet)} ephemeris rows from Miriade")
        df_comet.to_csv(miriade_filename, index=False)

## Get ephemerides from JPL Horizons

In [ ]:
for i, row in df_init.iterrows():
    print(f"{i}: {row['Comet']}")
    source = df_init.loc[i, 'Comet']
    name = " ".join(source.split())

    cadc_filename = f"data/cadc2/{name.replace('/', '_')}_cadc.csv"
    jplh_filename = f"data/jpl2/{name.replace('/', '_')}_jplhorizons.csv"
    if not os.path.exists(cadc_filename):
        print(f"  CADC file {cadc_filename} not found, skipping")
        continue
    
    if os.path.exists(jplh_filename):
        print(f"  JPL Horizons file {jplh_filename} already exists, skipping")
        continue
    
    df_cadc = pd.read_csv(cadc_filename)

    emin = Time(df_cadc['MJD'].min(), format='mjd')
    emax = Time(df_cadc['MJD'].max(), format='mjd')

    print(f"Have {name} from {emin.isot} to {emax.isot} ({emax.mjd-emin.mjd:.0f} days)")

    # query using 1-day time steps
    obj = Horizons(id=name, location='500', 
                   epochs=dict(start=emin.isot, stop=emax.isot, step='1d'))#, 

    jpltab = obj.ephemerides(extra_precision=True)
    fullname = jpltab['targetname'][0]
    print(f"Got positions from Horizons with {len(jpltab)} rows for {fullname}")

    # save to CSV
    jpltab.to_pandas().to_csv(jplh_filename, index=False)


## Analysis of the results

- Output useful (visible) observations as e.g., `./data/visible2/C_2004D1_visible.csv`

In [ ]:
counts = 0

# Directory with output data
outdir = "./data/visible2"
os.makedirs(outdir, exist_ok=True)


for i, row in df_init[0:].iterrows():
    is_cadc = False
    is_miriade = False
    is_jplh = False
    
    name = " ".join(row['Comet'].split())
    cadc_filename = f"data/cadc2/{name.replace('/', '_')}_cadc.csv"
    miriade_filename = f"data/miriade2/{name.replace('/', '_')}_miriade.csv"
    jplh_filename = f"data/jpl2/{name.replace('/', '_')}_jplhorizons.csv"
    try:
        df_cadc = pd.read_csv(cadc_filename)
        is_cadc = True
    except FileNotFoundError:
        print(f"{name}: CADC file not found")
        continue
    
    try:
        df_jplh = pd.read_csv(jplh_filename)
        is_jplh = True
    except FileNotFoundError:
        print(f"{name}: JPL Horizons file not found")
        # continue

    if is_jplh:
        df_merged = pd.merge(df_cadc, df_jplh, left_index=True, right_index=True, suffixes=('_cadc', '_jplh'))
    else:
        print(f"{name}: No JPL Horizons data to merge")
        continue
        
    # print(f"{name}: merged {len(df_merged)} rows")
    # display(df_merged[['Telescope/Instrument', 'Filter']].value_counts())

    # Keep only needed columns
    limits = df_limits[[c for c in ['Telescope/Instrument', 'Filter', 'Limiting', 'Saturation'] if c in df_limits.columns]].dropna(subset=['Telescope/Instrument'])

    # Keep only observations whose Telescope/Instrument exists in the limits
    valid_instr = set(limits['Telescope/Instrument'].unique())
    df_merged = df_merged[df_merged['Telescope/Instrument'].isin(valid_instr)].copy()

    # Attach limits to each observation; inner keeps only matched (Instrument, Filter)
    df_with_limits = df_merged.merge(limits, on=['Telescope/Instrument', 'Filter'], how='inner')

    # Condition: not saturated (Tmag >= Saturation) and not too faint (Tmag <= Limiting)
    cond = (df_with_limits['Tmag'] >= df_with_limits['Saturation']) & (df_with_limits['Tmag'] <= df_with_limits['Limiting'])

    df_visible = df_with_limits[cond]

    # Save visible observations 
    out = f"{name.replace('/', '_').replace(' ', '')}_visible.csv"
    out = os.path.join(outdir, out)
    df_visible.to_csv(out, sep=",")

    if len(df_visible) != 0:
        print(f"{i} - {name}: {len(df_visible)} visible observations")
        display(df_visible[['Telescope/Instrument', 'Filter']].value_counts())
        counts += len(df_visible)
    else:
        print(f"{name}: No visible observations")

    # if i == 2:
    #     break
print(f"Total visible observations: {counts}")

## Plots

In [ ]:
# name= "C/2023 H5"
# name = "C/2005 R4"
name = df_init.loc[13, 'Comet']
print(name)
cadc_filename = f"data/cadc2/{name.replace('/', '_')}_cadc.csv"
miriade_filename = f"data/miriade2/{name.replace('/', '_')}_miriade.csv"
try:
    df_cadc = pd.read_csv(cadc_filename)
except FileNotFoundError:
    print(f"{name}: CADC file not found")
    df_cadc = pd.DataFrame()
try:    df_miriade = pd.read_csv(miriade_filename)
except FileNotFoundError:
    print(f"{name}: Miriade file not found")
    df_miriade = pd.DataFrame()
df = pd.merge(df_cadc, df_miriade, left_index=True, right_index=True, suffixes=('_cadc', '_miriade'))
print(f"{name}: merged {len(df)} rows")
df

In [ ]:
# %%time 

# PS1 epoch range with epoch edges set to round days
jplh_filename = f"data/jpl2/{name.replace('/', '_')}_jplhorizons.csv"
df_jplh = pd.read_csv(jplh_filename)
fullname = df_jplh['targetname'][0]
print(f"Got positions from Horizons with {len(df_jplh)} rows for {fullname}")
df_jplh

In [ ]:
# undo RA-wrapping from 0-360 degrees
# add or subtract multiples of 360 to create a smooth path
njumps = 0
ra_deg = df_jplh['RA'].to_numpy(copy=True)

# Count wrap jumps in the original series (optional)
d = np.diff(ra_deg)
njumps = int((d > 180).sum() + (d < -180).sum())

# Unwrap using numpy (threshold = 180 deg)
ratab = np.rad2deg(np.unwrap(np.deg2rad(ra_deg), discont=np.deg2rad(180)))

# Shift to non-negative range
if ratab.min() < 0:
    ratab += 360.0 * np.ceil(-ratab.min() / 360.0)

# plt.rcParams.update({"font.size":14})
plt.figure(1, (12, 6))
plt.plot(ratab, df_jplh['DEC'])

# plot every 30 days as a dot to see movement
mstep = int(30 / (df_jplh['datetime_jd'][1] - df_jplh['datetime_jd'][0]) + 0.5)
plt.plot(ratab[::mstep], df_jplh['DEC'][::mstep], '.', color="tab:blue", label="Monthly positions")

# show positions at the times of observations
jpl_mjd = Time(df_jplh['datetime_jd'], format='jd').mjd
ra_interp = np.interp(df['MJD'], jpl_mjd, ratab)
dec_interp = np.interp(df['MJD'], jpl_mjd, df_jplh['DEC'])

df_instr = df[['Telescope/Instrument', 'Filter']].value_counts()
df_instr = df_instr[df_instr > 10]
for (instr, filt), count in df_instr.items():
    cond = (df['Telescope/Instrument'] == instr) & (df['Filter'] == filt)
    plt.plot(ra_interp[cond], dec_interp[cond], '*', markersize=10,
             label=f"{instr} {filt} ({count})")

plt.xlabel("Unwrapped RA [deg]")
plt.ylabel("Dec [deg]")
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.title(f"{fullname} from {emin.isot[:10]} to {emax.isot[:10]}")
plt.tight_layout()
plt.grid(linestyle='--', alpha=0.5)
# if saveplots:
#     plt.savefig("sky_path.png", facecolor="white")
plt.show()

In [ ]:
phase_interp = np.interp(df_cadc['MJD'], jpl_mjd, df_jplh['alpha']) 
# Nmag
nmag_interp = np.interp(df_cadc['MJD'], jpl_mjd, df_jplh['Nmag'])
# distance from the Sun
r_interp = np.interp(df_cadc['MJD'], jpl_mjd, df_jplh['r'])

# plt.rcParams.update({"font.size":14})
plt.figure(1,(12,6))
# plt.plot(ratab, jpltab['DEC'])

df_instr = df_cadc[['Telescope/Instrument', 'Filter']].value_counts()
df_instr = df_instr[df_instr > 1]
for (instr, filt), count in df_instr.items():
    cond = (df_cadc['Telescope/Instrument'] == instr) & (df_cadc['Filter'] == filt)
    plt.plot(
        # phase_interp[cond], 
        r_interp[cond],
         nmag_interp[cond], 
        '*', markersize=10,
        label=f"{instr} {filt} ({count})")
# plt.plot(ra_interp, dec_interp, 'o', color="tab:orange", label=f"{len(df)} observations")

plt.xlabel("The distance from the Sun [au]")
plt.ylabel("Nmag [mag]")
# plt.xlim(left=0)
# limit legend by 30 entries
handles, labels = plt.gca().get_legend_handles_labels()
if len(handles) > 30:
    handles = handles[:30]
    labels = labels[:30]
    plt.gca().legend(handles, labels, bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=8)
# plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=8)

plt.title(f"{fullname} from {emin.isot[:10]} to {emax.isot[:10]}")
plt.tight_layout()
plt.grid(linestyle='--', alpha=0.5)
# if saveplots:
#     plt.savefig("sky_path.png", facecolor="white")
plt.show()
